In [1]:
# 通过意见表的意见，遍历文档找到仅出现一次的case，生成一个mark csv，即标志为科研通过code自动替换的。方便后面进行人工审核。
import csv
import docx
import xlrd
from tqdm import tqdm
import utils

In [2]:
# config
xlsx_path = '/mnt/c/Users/gray/seadrive_root/mail_1/我的资料库/mine/STANDARD/第三次会议/ALL.xlsx'

# in_path = '/home/gray/desktop/STANDARD/test2.docx' # example
in_path = '/mnt/c/Users/gray/seadrive_root/mail_1/我的资料库/mine/STANDARD/第三次会议/11.11.docx'

out_path = '/mnt/c/Users/gray/seadrive_root/mail_1/我的资料库/mine/STANDARD/第三次会议/out.docx'

doc = docx.Document(in_path)

In [3]:
# init replace dicts
replace = {}
replace_table = {}

workbook = xlrd.open_workbook(filename=xlsx_path)
worksheet = workbook.sheet_by_name(workbook.sheet_names()[0])

In [4]:
for i in range(worksheet.nrows):
    if i <= 3:
        continue
    row = worksheet.row_values(i)
    sec_num = utils.get_sec_num(str(row[1]))
    
    pre_result = row[7]  # 0 -> deny, 1-> yes
    discuss = row[8]  # discuss flag
    global_index = row[10]
    
    if discuss != 1 and pre_result != 0:  # 即 没有需要讨论 且 没有被拒绝
        if sec_num is not None:  # 
            # tmp solution: self-increment for a new added-in section
            tmp_s = [int(x) for x in sec_num.split('.')]
            tmp_s[0] += 1

            sec_num = ".".join([str(x) for x in tmp_s])
           
            
            
            if sec_num not in replace.keys():
                replace[sec_num] = []
            replace[sec_num].append((row[2],row[3],row[10]))  # sec_num : (before, after, index)
        else:  # for table
            replace_table[row[2]] = (row[3],row[10])  # before : (after, index)

In [5]:
[v[0][2] for v in replace.values()]

[3.0,
 9.0,
 25.0,
 26.0,
 28.0,
 33.0,
 35.0,
 36.0,
 37.0,
 38.0,
 39.0,
 40.0,
 41.0,
 48.0,
 169.0,
 174.0,
 183.0,
 184.0,
 193.0,
 196.0,
 198.0,
 203.0,
 206.0,
 208.0,
 218.0,
 222.0,
 230.0,
 237.0,
 249.0,
 276.0,
 279.0,
 292.0,
 300.0,
 303.0,
 305.0,
 308.0,
 310.0,
 311.0,
 313.0,
 314.0,
 315.0,
 317.0,
 324.0,
 334.0,
 335.0,
 341.0,
 344.0,
 346.0,
 348.0,
 353.0,
 354.0,
 356.0,
 357.0,
 358.0,
 365.0,
 366.0,
 367.0,
 371.0,
 374.0,
 375.0,
 376.0,
 377.0,
 378.0,
 380.0,
 383.0,
 385.0,
 387.0,
 389.0,
 390.0,
 394.0,
 396.0,
 397.0,
 398.0,
 400.0,
 402.0,
 403.0,
 405.0,
 416.0,
 418.0,
 419.0,
 423.0,
 426.0,
 427.0,
 431.0,
 432.0,
 433.0,
 434.0,
 436.0,
 439.0,
 440.0,
 443.0,
 444.0,
 445.0,
 446.0,
 450.0,
 451.0,
 452.0,
 454.0,
 460.0,
 471.0,
 477.0,
 478.0,
 479.0,
 480.0,
 482.0,
 483.0,
 484.0,
 485.0,
 486.0,
 488.0,
 489.0,
 490.0,
 491.0,
 493.0,
 495.0,
 496.0,
 497.0,
 498.0,
 499.0,
 500.0,
 501.0,
 502.0,
 503.0,
 505.0,
 507.0,
 509.0,
 510.0,


In [6]:
print("len of replace in text",len(replace))
print("len of replace in text",len(replace_table))

len of replace in text 146
len of replace in text 125


In [7]:
# replace里存的是xlsx里的替换规则
replace

{'5.1.1': [('表1描述了为阵列提供的配置文件。', '表1描述了阵列的相关配置文件。', 3.0)],
 '5.1.2': [('（此处漏了翻译）', '4.1.2 概述', 9.0),
  ('使用外部存储或是本地与外部混合的存储的系统称为存储虚拟化。', '使用外部存储或是本地与外部混合的存储的系统称为存储虚拟器。', 10.0),
  ('插入到底板或是母板的系统应该使用主机硬件RAID。',
   '插入到背板或是主板的系统应该使用“存储管理技术规范，第7部分 主机元素1.8.0版本4中的8 主机硬件RAID控制器配置文件。',
   11.0),
  ('阵列配置文件包含一个多计算机系统通讯接口模块对象，通过这个对象阵列将被看出一个整体，',
   '阵列配置文件包含一个从整体上代表此队列的CIM_ComputerSystems对象，',
   12.0),
  ('包含了：存储扩展盘，代表了物理存储设备；存储池，它集合了扩展盘、提供定位和服务质量的设备；存储卷，是从存储池到逻辑设备的定位。',
   '包含了：StorageExtents，代表了物理存储设备；StoragePools，将区段集合在一起，并提供分配和服务质量的设置；StroageVolumes，代表了从池里分配的逻辑设备。',
   13.0),
  ('目标端口通过那些对主机有用的逻辑单元数(LUN)来组成配置文件模型的端口部分(例如，iSCSI的光纤通道)。',
   '目标端口组件配置文件对端口（例如，是光纤通道还是iSCSI）进行建模，逻辑单元号(LUN)通过端口为主机所使用。',
   14.0),
  ('该实例通过元素功能与顶级阵列计算机系统相关联。', '该实例通过ElementCapabilities与顶级阵列计算机系统相关联。', 15.0),
  ('定位、软硬件版本以及阵列管理界面的访问。', '定位、软硬件版本以及阵列管理接口的访问。', 16.0),
  ('基本的阵列配置文件仅仅包含计算机系统通讯接口模块对象', '基本的阵列配置文件仅仅包含CIM_ComputerSystem对象', 17.0),
  ('这个对象关联到其它的子配置文件和包。', '这个对象通过一套关联集合与其它的子配置文件和包关联在一起。', 18.0),
  ('这种子配置文

In [8]:
# 搜索段落 每个只关联自己section内部（即section内部只出现一次）
found = {}  # 格式 "before : [show_counter, index]"
# search in paragraphs
now_sec = ''
for para in tqdm(list(doc.paragraphs)):
    # update sec_num
    sec_num = utils.get_sec_num(para.text)
    if sec_num is not None:
        now_sec = sec_num
    if now_sec == '':
        continue
    if now_sec not in replace.keys():
        continue
    
    for a,b,index in replace[now_sec]:
        if a != '' and a in para.text:
            if a not in found.keys():
                found[a] = [0,b,index]
            
            found[a][0] += 1

100%|██████████████████████████████████████████████████████████████████████| 12046/12046 [00:00<00:00, 81430.59it/s]


In [9]:
# 搜索表格
found_table = {}  # 格式 "before : [counter, afer, index]"
for table_index in tqdm(range(len(list(doc.tables)))):
# for table_index in tqdm(range(len(list(doc.tables)))[10:]):
    table = doc.tables[table_index]
    for row in table.rows:
        for cell in row.cells:
            for para in cell.paragraphs:
                for a,b in replace_table.items():
                    if a!='' and a in para.text:
                        if a not in found_table.keys():
                            found_table[a] = [0]+list(b)
                        found_table[a][0] += 1
# print(found_table)

100%|█████████████████████████████████████████████████████████████████████████████| 723/723 [00:29<00:00, 24.35it/s]


In [10]:
# search有匹配的数量
print(len(found))
print(len(found_table))

1
42


In [11]:
[v for v in found_table.values()]

[[1331, '可选的', 168.0],
 [11, 'Job Control', 519.0],
 [12, 'Indications', 520.0],
 [36, '描述和注意事项', 163.0],
 [1, '代表整个阵列的顶级系统，与RegisteredProfile相关联。', 31.0],
 [1, 'CQL-计算机系统状态的改变。PreviousInstance是可选的，但是可以由配置文件的实现提供。', 42.0],
 [1, 'CQL——计算机系统ElementName的改变。PreviousInstance是可选的，但是可以由配置文件的实现提供。', 43.0],
 [1, 'StorageVolume已降级。', 45.0],
 [1, 'StorageVolume出现故障。', 46.0],
 [1, 'StorageVolume已恢复正常服务。', 47.0],
 [43, '属性', 160.0],
 [188, '标志位', 161.0],
 [1, '该阵列的唯一标识，例如IP地址、PQDN或DNS名字。', 61.0],
 [1, '条件要求:如果提供了OtherIdentifyingInfo，则为必填项。', 62.0],
 [1, '阵列的整体状态', 63.0],
 [1, '标识该计算机系统专用于存储阵列的操作', 64.0],
 [1, '阵列所有者的联系信息', 65.0],
 [2, 'ImplementationCapabilities。', 78.0],
 [2, '具有ImplementationCapabilities的符合的阵列计算机系统。', 79.0],
 [1,
  '有效的字符串值为“none”|“SNIA:VolumeView”|“SNIA:磁DiskDriveView”|“SNIA:ExposedView”|“SNIA:MaskingMappingView”|“SNIA:MappingProtocolControllerView”|“SNIA:StoragePoolView”|“SNIA:ReplicaPairView”。',
  93.0],
 [2, '关联设备的地址（例如逻辑单元号），应被格式化为不分隔的大写十六进制数字，没有前导0x。', 119.0],
 [2, '赋予引用的逻辑

In [12]:
# 找到只出现一次的
indexes = []
for k,v in found.items():
    if v[0] ==  1:
        indexes.append(v[2])

for k,v in found_table.items():
    if v[0] <= 1:
        indexes.append(v[2])
        
print(len(indexes))
print(indexes[:10])

30
[85.0, 31.0, 42.0, 43.0, 45.0, 46.0, 47.0, 61.0, 62.0, 63.0]


In [14]:
sorted(indexes)

[31.0,
 42.0,
 43.0,
 45.0,
 46.0,
 47.0,
 61.0,
 62.0,
 63.0,
 64.0,
 65.0,
 85.0,
 93.0,
 121.0,
 248.0,
 253.0,
 254.0,
 255.0,
 257.0,
 263.0,
 264.0,
 265.0,
 266.0,
 267.0,
 268.0,
 273.0,
 274.0,
 291.0,
 521.0,
 586.0]

In [15]:
indexes_sorted = sorted([int(i) for i in indexes])
data = []
for i in range(int(indexes_sorted[-1])+1):
    if i in indexes_sorted:
        data.append([1])
    else:
        data.append([])

In [16]:
data

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [1],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [1],
 [1],
 [1],
 [1],
 [1],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [1],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [1],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [1],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 

In [17]:
# please merge this detect results to xlsx manually
with open('/home/gray/desktop/detect.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerows(data)

In [18]:
# -------------------------------below is for testing------------------------------------------------